<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/---PDF-20250403/blob/main/%E3%80%90note%E5%85%AC%E9%96%8B%E3%80%91Google_Colab%E3%81%A7%E3%81%99%E3%81%90%E3%81%AB%E4%BD%BF%E3%81%88%E3%82%8BNCBI%E8%AB%96%E6%96%87%E6%A4%9C%E7%B4%A2%E3%83%84%E3%83%BC%E3%83%AB%E2%91%A0%E3%82%B7%E3%83%B3%E3%83%97%E3%83%AB%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NCBI論文検索ツール


使い方

Google Colabで新しいノートブックを開く
提供したコードをコピー＆ペースト
セルを実行
表示されるフォームに検索条件を入力

キーワード（例：cancer therapy）
著者名（例：Smith J）
ジャーナル名（例：Nature）
出版年の範囲（例：2020～2023）
「検索」ボタンをクリック
検索結果から論文URLをクリックするか、CSVに保存

※2025/04/11APIテスト済み　APIの仕様変更等によりデータ抽出が出来なくなる場合がございます。

In [4]:
# 簡易NCBI論文検索ツール
# Google Colab用シンプルバージョン
# ユーザーフレンドリーな絞り込み検索 & URL取得機能

# 必要なライブラリのインストール
# !pip install pandas requests ipywidgets matplotlib seaborn tqdm

import requests
import xml.etree.ElementTree as ET
import pandas as pd
import time
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import ipywidgets as widgets

class SimpleNCBISearch:
    """
    NCBIの論文を簡単に検索・取得するためのシンプルなクラス
    """

    def __init__(self):
        self.base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
        self.current_articles = []  # 現在の検索結果

    def search_pubmed(self, query_params):
        """
        PubMedで検索を実行

        Parameters:
        query_params (dict): 検索パラメータ

        Returns:
        list: 論文情報のリスト
        """
        # 検索クエリの構築
        query_parts = []

        # キーワード
        if query_params["keywords"]:
            query_parts.append(f"({query_params['keywords']})")

        # 著者
        if query_params["author"]:
            query_parts.append(f"{query_params['author']}[Author]")

        # ジャーナル
        if query_params["journal"]:
            query_parts.append(f"{query_params['journal']}[Journal]")

        # 出版年
        if query_params["year_from"] and query_params["year_to"]:
            query_parts.append(f"{query_params['year_from']}:{query_params['year_to']}[pdat]")
        elif query_params["year_from"]:
            query_parts.append(f"{query_params['year_from']}[pdat]")
        elif query_params["year_to"]:
            query_parts.append(f":{query_params['year_to']}[pdat]")

        # 最終クエリ
        final_query = " AND ".join(query_parts)

        # 検索実行
        search_url = f"{self.base_url}esearch.fcgi"
        search_params = {
            "db": "pubmed",
            "term": final_query,
            "retmax": query_params["max_results"],
            "usehistory": "y",
            "retmode": "xml"
        }

        try:
            search_response = requests.get(search_url, params=search_params)
            search_response.raise_for_status()

            search_root = ET.fromstring(search_response.content)

            # 検索結果カウント
            count_elem = search_root.find(".//Count")
            if count_elem is None or int(count_elem.text) == 0:
                print("検索結果が見つかりませんでした。検索条件を変更してみてください。")
                return []

            # WebEnvとQueryKey取得
            web_env = search_root.find(".//WebEnv").text
            query_key = search_root.find(".//QueryKey").text
            count = int(count_elem.text)

            print(f"検索結果: {count}件 (最大{query_params['max_results']}件を表示)")

            # 論文詳細を取得
            fetch_url = f"{self.base_url}efetch.fcgi"
            fetch_params = {
                "db": "pubmed",
                "query_key": query_key,
                "WebEnv": web_env,
                "retmax": query_params["max_results"],
                "retmode": "xml",
                "rettype": "abstract"
            }

            fetch_response = requests.get(fetch_url, params=fetch_params)
            fetch_response.raise_for_status()

            fetch_root = ET.fromstring(fetch_response.content)

            # 論文データを抽出
            articles = []

            for article_elem in fetch_root.findall(".//PubmedArticle"):
                # PMID
                pmid_elem = article_elem.find(".//PMID")
                pmid = pmid_elem.text if pmid_elem is not None else "Unknown"

                # タイトル
                title_elem = article_elem.find(".//ArticleTitle")
                title = title_elem.text if title_elem is not None else "No title available"

                # 著者
                authors = []
                author_list = article_elem.findall(".//Author")

                for author in author_list:
                    last_name = author.find("LastName")
                    fore_name = author.find("ForeName")

                    if last_name is not None and fore_name is not None:
                        authors.append(f"{last_name.text} {fore_name.text}")
                    elif last_name is not None:
                        authors.append(last_name.text)

                # ジャーナル
                journal_elem = article_elem.find(".//Journal/Title")
                journal = journal_elem.text if journal_elem is not None else "Unknown journal"

                # 出版年月
                year = "Unknown"
                month = "Unknown"

                year_elem = article_elem.find(".//PubDate/Year")
                if year_elem is not None:
                    year = year_elem.text

                month_elem = article_elem.find(".//PubDate/Month")
                if month_elem is not None:
                    month = month_elem.text

                # DOI
                doi = None
                for id_elem in article_elem.findall(".//ArticleId"):
                    if id_elem.get("IdType") == "doi":
                        doi = id_elem.text
                        break

                # PubMed URL
                pubmed_url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"

                # 抄録
                abstract_text = ""
                abstract_elems = article_elem.findall(".//AbstractText")
                for abstract_elem in abstract_elems:
                    if abstract_elem.text:
                        abstract_text += abstract_elem.text + " "

                if not abstract_text:
                    abstract_text = "No abstract available"

                # 論文情報を辞書に格納
                article_info = {
                    "pmid": pmid,
                    "title": title,
                    "authors": authors,
                    "journal": journal,
                    "year": year,
                    "month": month,
                    "doi": doi,
                    "pubmed_url": pubmed_url,
                    "abstract": abstract_text
                }

                articles.append(article_info)

            self.current_articles = articles
            return articles

        except requests.exceptions.RequestException as e:
            print(f"APIリクエストエラー: {e}")
            return []
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            return []

    def export_to_csv(self, articles, filename="pubmed_results.csv"):
        """
        検索結果をCSVファイルに保存

        Parameters:
        articles (list): 論文情報のリスト
        filename (str): 保存するファイル名
        """
        if not articles:
            print("エクスポートするデータがありません。")
            return

        data = []
        for article in articles:
            row = {
                "PMID": article["pmid"],
                "Title": article["title"],
                "Authors": ", ".join(article["authors"]) if article["authors"] else "",
                "Journal": article["journal"],
                "Year": article["year"],
                "Month": article["month"],
                "DOI": article["doi"] if article["doi"] else "",
                "PubMed URL": article["pubmed_url"],
                "Abstract": article["abstract"]
            }
            data.append(row)

        df = pd.DataFrame(data)
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"データを {filename} に保存しました")

        # Google Colabでダウンロードリンクを表示
        try:
            from google.colab import files
            files.download(filename)
        except:
            pass

    def plot_year_distribution(self, articles):
        """
        出版年の分布をプロット

        Parameters:
        articles (list): 論文情報のリスト
        """
        if not articles:
            print("表示するデータがありません。")
            return

        # 年を抽出
        years = []
        for article in articles:
            if article["year"] and article["year"] != "Unknown":
                try:
                    year = int(article["year"])
                    years.append(year)
                except ValueError:
                    continue

        if not years:
            print("年データがありません。")
            return

        # 年ごとの論文数をカウント
        year_counts = pd.Series(years).value_counts().sort_index()

        # グラフ作成
        plt.figure(figsize=(10, 5))
        bars = sns.barplot(x=year_counts.index, y=year_counts.values)

        # 棒グラフに値を表示
        for i, v in enumerate(year_counts.values):
            bars.text(i, v + 0.1, str(v), ha='center')

        plt.title("出版年別論文数")
        plt.xlabel("出版年")
        plt.ylabel("論文数")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# インタラクティブ検索UIの実装
def create_search_ui():
    """
    ユーザーフレンドリーな検索インターフェースを作成
    """
    # NCBIサーチャーのインスタンス化
    searcher = SimpleNCBISearch()

    # フォーム要素の作成
    keyword_input = widgets.Text(
        value='',
        placeholder='キーワード（例: cancer therapy）',
        description='キーワード:',
        layout=widgets.Layout(width='100%')
    )

    author_input = widgets.Text(
        value='',
        placeholder='例: Smith J',
        description='著者名:',
        layout=widgets.Layout(width='100%')
    )

    journal_input = widgets.Text(
        value='',
        placeholder='例: Nature',
        description='ジャーナル:',
        layout=widgets.Layout(width='100%')
    )

    year_from = widgets.Text(
        value='',
        placeholder='例: 2020',
        description='出版年（開始）:',
        layout=widgets.Layout(width='48%')
    )

    year_to = widgets.Text(
        value='',
        placeholder='例: 2023',
        description='出版年（終了）:',
        layout=widgets.Layout(width='48%')
    )

    max_results = widgets.IntSlider(
        value=20,
        min=5,
        max=100,
        step=5,
        description='表示件数:',
        layout=widgets.Layout(width='100%')
    )

    search_button = widgets.Button(
        description='検索',
        button_style='primary',
        icon='search',
        layout=widgets.Layout(width='30%')
    )

    export_button = widgets.Button(
        description='CSVに保存',
        button_style='success',
        icon='download',
        layout=widgets.Layout(width='30%')
    )

    plot_button = widgets.Button(
        description='年別グラフ',
        button_style='info',
        icon='bar-chart',
        layout=widgets.Layout(width='30%')
    )

    # 結果表示領域
    output_area = widgets.Output()

    # UIのレイアウト
    display(widgets.HTML("<h2>NCBI/PubMed 簡易論文検索ツール</h2>"))
    display(widgets.HTML("<p>少なくとも1つの検索条件を入力してください</p>"))

    search_form = widgets.VBox([
        keyword_input,
        widgets.HBox([author_input, journal_input]),
        widgets.HBox([year_from, year_to]),
        max_results,
        widgets.HBox([search_button, export_button, plot_button])
    ])

    display(search_form)
    display(output_area)

    # ボタンイベントの処理
    def on_search_button_clicked(b):
        with output_area:
            clear_output()

            # 検索パラメータの取得
            params = {
                "keywords": keyword_input.value.strip(),
                "author": author_input.value.strip(),
                "journal": journal_input.value.strip(),
                "year_from": year_from.value.strip(),
                "year_to": year_to.value.strip(),
                "max_results": max_results.value
            }

            # 少なくとも1つのパラメータが必要
            if not any([params["keywords"], params["author"], params["journal"], params["year_from"], params["year_to"]]):
                print("少なくとも1つの検索条件を入力してください。")
                return

            print("検索中...")
            articles = searcher.search_pubmed(params)

            if not articles:
                return

            # 検索結果の表示
            print("\n検索結果:")

            result_table = pd.DataFrame([
                {
                    "タイトル": article["title"],
                    "著者": ", ".join(article["authors"][:3]) + ("..." if len(article["authors"]) > 3 else ""),
                    "ジャーナル": article["journal"],
                    "年": article["year"],
                    "URL": f"<a href='{article['pubmed_url']}' target='_blank'>PubMed</a>"
                }
                for article in articles
            ])

            # HTMLとして結果テーブルを表示
            display(HTML(result_table.to_html(escape=False, index=False)))

            # 各論文の詳細表示
            for i, article in enumerate(articles):
                if i < 5:  # 最初の5件だけ詳細表示
                    display(HTML(f"""
                    <div style="margin-top: 20px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                        <h3>{article['title']}</h3>
                        <p><b>著者:</b> {', '.join(article['authors'])}</p>
                        <p><b>ジャーナル:</b> {article['journal']} ({article['year']})</p>
                        <p><b>URL:</b> <a href="{article['pubmed_url']}" target="_blank">{article['pubmed_url']}</a></p>
                        <p><b>抄録:</b> {article['abstract'][:300]}...</p>
                    </div>
                    """))

    def on_export_button_clicked(b):
        with output_area:
            if not searcher.current_articles:
                print("エクスポートするデータがありません。先に検索を実行してください。")
                return

            # ファイル名の生成（検索条件を含む）
            filename_parts = []
            if keyword_input.value:
                filename_parts.append(keyword_input.value.replace(" ", "_"))
            if author_input.value:
                filename_parts.append(author_input.value.replace(" ", "_"))
            if journal_input.value:
                filename_parts.append(journal_input.value.replace(" ", "_"))

            if filename_parts:
                filename = f"pubmed_{'_'.join(filename_parts)}.csv"
            else:
                filename = "pubmed_results.csv"

            searcher.export_to_csv(searcher.current_articles, filename)

    def on_plot_button_clicked(b):
        with output_area:
            if not searcher.current_articles:
                print("表示するデータがありません。先に検索を実行してください。")
                return

            searcher.plot_year_distribution(searcher.current_articles)

    # ボタンのクリックイベントを設定
    search_button.on_click(on_search_button_clicked)
    export_button.on_click(on_export_button_clicked)
    plot_button.on_click(on_plot_button_clicked)

# Colabで実行されているか確認
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

# 必要なライブラリをインストール
if IN_COLAB:
    try:
        import seaborn
    except ImportError:
        print("必要なライブラリをインストールしています...")
        !pip install -q pandas requests ipywidgets matplotlib seaborn tqdm
        print("インストール完了")

# ツールの説明を表示
def show_tool_intro():
    display(HTML("""
    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px;">
        <h1>PubMed/NCBI 簡易論文検索ツール</h1>
        <p><b>このツールでできること:</b></p>
        <ul>
            <li>キーワード、著者名、ジャーナル名、出版年で論文を検索</li>
            <li>検索結果からPubMed URLを簡単に取得</li>
            <li>検索結果をCSVファイルに保存</li>
            <li>出版年別の論文数グラフを表示</li>
        </ul>
        <p><b>使い方:</b></p>
        <ol>
            <li>検索条件を入力（少なくとも1つは必要）</li>
            <li>「検索」ボタンをクリック</li>
            <li>結果から論文のURLにアクセス、またはCSVに保存</li>
        </ol>
    </div>
    """))

# メイン実行部分
def main():
    show_tool_intro()
    create_search_ui()

# 実行
if __name__ == "__main__" or IN_COLAB:
    main()

HTML(value='<h2>NCBI/PubMed 簡易論文検索ツール</h2>')

HTML(value='<p>少なくとも1つの検索条件を入力してください</p>')

Output()